# #413 Box section following path
<i></i>
***

In [ ]:
# Options
INCREMENT = 0.1       # Sections defined at parametric distances along the path
segment_length = 0    # Optionally define a specific segment length, ehen this is non zero the path will be split into segments of this length
create_path = True    # option to create the reference path
path_name = "Path1"   # name of path to be used (either created or existing in the model.)

# Box section dimensions
top_flange_breadth_between_webs = 6
bottom_flange_breadth = 5
outstand_breadth = 3
web_height = 3

***

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()
db = lusas.database()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

import numpy as np
import math
import copy

In [ ]:
if create_path:
    ref_path = db.createReferencePath(path_name)
    poly = ref_path.getDefn()
    poly.deleteAllLines()
    poly.startSplineCrds()
    poly.addSplineFitCrds(0.0, 0.0, 0.0)
    poly.addSplineFitCrds(20.0, 3.0, 0.0)
    poly.addSplineFitCrds(40.0, 9.0, 0.0)
    poly.addSplineFitCrds(60.0, 18.0, 0.0)
    poly.addSplineFitCrds(80.0, 36.0, 0.0)
    poly.addSplineFitCrds(100.0, 72.0, 0.0)
    poly.addSplineFitCrds(120.0, 144.0, 0.0)
    poly.endSplineCrds()
    poly.setFacetData(20, 6, -1.0, 3.0, -1.0)
    poly.setSmoothing(False)
    poly.setFilletType(0, True)
    ref_path.setDistanceStartOfPath(0.0)
    ref_path.setTransverseDirection("Perpendicular")

In [ ]:
class Point:
    
    def __init__(self, x:float, y:float, z:float):
        self.X = x
        self.Y = y
        self.Z = z

    def __repr__(self) -> str:
        return f"({self.X}, {self.Y})"

In [ ]:
class BoxSection:

    def __init__(self, bf_width, tf_width, outstand_width, web_depth):
        # Top flange
        self.P1 = Point(0.0,  tf_width / 2 + outstand_width, 0.0)
        self.P2 = Point(0.0,  tf_width / 2, 0.0)
        self.P3 = Point(0.0, -tf_width / 2, 0.0)
        self.P4 = Point(0.0, -tf_width / 2 - outstand_width, 0.0)
        # Bottom flange
        self.P5 = Point(0.0,  bf_width / 2, -web_depth)
        self.P6 = Point(0.0, -bf_width / 2, -web_depth)

        self.model_points : list[IFPoint]
        self.model_lines : list[IFLine]

    def get_points(self):
        return [self.P1, self.P2, self.P3, self.P4, self.P5, self.P6]
    
    def translate(self, x:float, y:float, z:float):
        for p in self.get_points():
            p.X += x
            p.Y += y
            p.Z += z

    def rotate(self, origin, theta_z):
        for p in self.get_points():
            dx = 0 # section should have been translated to x
            dy = p.Y-origin[1]
            x1 = (dx * math.cos(theta_z) - dy * math.sin(theta_z))
            y1 = (dx * math.sin(theta_z) + dy * math.cos(theta_z))
            p.X += (x1-dx)
            p.Y += (y1-dy)


    def create_model_points(self):
        self.model_points = [Helpers.create_point(p.X, p.Y, p.Z) for p in self.get_points()]

    def create_model_lines(self):
        self.model_lines = []
        # Top Flange
        for i in range(3):
            self.model_lines.append(Helpers.create_line_from_points(self.model_points[i], self.model_points[i+1]))

        # Webs
        self.model_lines.append(Helpers.create_line_from_points(self.model_points[1], self.model_points[4]))
        self.model_lines.append(Helpers.create_line_from_points(self.model_points[2], self.model_points[5]))

        # Bottom flange        
        self.model_lines.append(Helpers.create_line_from_points(self.model_points[4], self.model_points[5]))  
     

In [ ]:
base_section = BoxSection(bottom_flange_breadth, top_flange_breadth_between_webs, outstand_breadth, web_height)

In [ ]:
# Reference path defining the bridge alignment
ref_path : IFReferencePath = win32.CastTo( db.getObject("Reference Path", "Path1"), "IFReferencePath" )
poly = ref_path.getDefn()

In [ ]:
# Start of the path
xStart = 0
yStart = 0
zStart = 0
poly.getStartCoord(xStart, yStart, zStart)

In [ ]:
section_list :list[BoxSection] = []

In [ ]:
# Determine the increment at which to create a section
if (segment_length > 0):
    inc = segment_length / ref_path.getDefn().getLength()
else:
    inc = INCREMENT

In [ ]:

for x in np.arange(0.0, 1.01, inc):
     # Translation
    position = poly.getInterpolatedPosition(x)
    xOffset = position[0] - xStart
    yOffset = position[1] - yStart
    zOffset = position[2] - zStart

    # Rotations
    tangent = poly.getInterpolatedTangent(x)
    theta_z = np.arccos(tangent[0])
    if(tangent[1] < 0):
        theta_z *= -1

    # New section 
    new_section = copy.deepcopy(base_section)
    # move to the parametric distance x
    new_section.translate(xOffset, yOffset, zOffset)
    # rotate to tangent
    new_section.rotate(position, theta_z)
    # create geometry
    new_section.create_model_points()
    new_section.create_model_lines()
    # save section
    section_list.append(new_section)

In [ ]:
# Join the sections
for ip, sp in enumerate(base_section.get_points()):

    objs = lusas.newObjectSet()

    for s in section_list:
        objs.add(s.model_points[ip])

    geom_data = lusas.geometryData().setAllDefaults()
    geom_data.setCreateMethod("spline")
    geom_data.useSelectionOrder(True)
    geom_data.setLowerOrderGeometryType("points")
    newObjs = objs.createLine(geom_data)
    newObjs.add(objs)
    geom_data.setAllDefaults()
    splitLines = newObjs.splitLine(geom_data)

    assert splitLines.count("Line") == len(section_list) - 1

In [ ]:
# Surfaces

for iSection in range(len(section_list)-1):

    s1 = section_list[iSection]
    s2 = section_list[iSection+1]

    long_lines = []
    for iLine in range(len(base_section.get_points())):
        long_lines.append(Helpers.get_line_between_points(s1.model_points[iLine], s2.model_points[iLine]))

    # Top Flange
    for i in range(3):
        Helpers.create_surface_from_lines([long_lines[i], s2.model_lines[i], long_lines[i+1], s1.model_lines[i]])

    # Webs
    Helpers.create_surface_from_lines([long_lines[1], s2.model_lines[-3], long_lines[-2], s1.model_lines[-3]])
    Helpers.create_surface_from_lines([long_lines[2], s2.model_lines[-2], long_lines[-1], s1.model_lines[-2]])

    # Bottom flange
    Helpers.create_surface_from_lines([long_lines[-2], s2.model_lines[-1], long_lines[-1], s1.model_lines[-1]])    
